In [24]:
import tensorflow as tf
from nltk.tokenize import word_tokenize
import numpy as np
import functools
from itertools import chain
from sklearn import model_selection, metrics
import pprint

from process_data import load_entities, load_task, vectorize, vectorize_kv_pairs, save_pickle, load_pickle, load_kv_pairs
# from net.memn2n import MemN2N
from net.memn2n_kv import MemN2N_KV, add_gradient_noise

pp = pprint.PrettyPrinter()

In [2]:
train_data = load_pickle('mov_task1_qa_pipe_train.pickle')
test_data = load_pickle('mov_task1_qa_pipe_test.pickle')
kv_pairs = load_pickle('mov_kv_pairs.pickle')
stopwords = load_pickle('mov_stopwords.pickle')
entities = load_pickle('mov_entities.pickle')

load mov_task1_qa_pipe_train.pickle
load mov_task1_qa_pipe_test.pickle
load mov_kv_pairs.pickle
load mov_stopwords.pickle
load mov_entities.pickle


In [3]:
word_kv_pairs = []
for kv in kv_pairs:
    words = []
    for ent in kv:
        words += ent.split(' ')
    word_kv_pairs.append(words)
# save_pickle(word_kv_pairs, 'mov_word_kv_pairs.pickle')

In [16]:
def get_kv_indices(data, word_kv_pairs, stopwords):
    kv_indices = []
    for i, (_, q, _) in enumerate(data):
        if i%100 == 0: print(i, '/', len(data))
        ind = []
        for w in q:
            if w in stopwords: continue
            for j, kv in enumerate(word_kv_pairs):
                if w in kv:
                    ind.append(j)
                    break
        kv_indices.append(ind)
#         if i > 100: break
    return kv_indices

def get_kv_indices2(data, kv_pairs, stopwords, entities):
    kv_indices = []
    for i, (_, q, _) in enumerate(train_data):
        if i % 10 == 0: 
            print(i, '/', len(train_data))
        ind = []
        question = ' '.join(q)
        for j, kv in enumerate(kv_pairs):
#             if w in kv:
            for ent in kv:
                if not ent in stopwords and ent in entities:
                    if ent in question:
                        print(ent, 'in', question)
                        ind.append(j)
                        break
        kv_indices.append(ind)
        if i > 30: break
    return kv_indices

train_kv_indices = get_kv_indices(train_data, word_kv_pairs, stopwords)
test_kv_indices = get_kv_indices(test_data, word_kv_pairs, stopwords)

0 / 96185
100 / 96185
200 / 96185
300 / 96185
400 / 96185
500 / 96185
600 / 96185
700 / 96185
800 / 96185
900 / 96185
1000 / 96185
1100 / 96185
1200 / 96185
1300 / 96185
1400 / 96185
1500 / 96185
1600 / 96185
1700 / 96185
1800 / 96185
1900 / 96185
2000 / 96185
2100 / 96185
2200 / 96185
2300 / 96185
2400 / 96185
2500 / 96185
2600 / 96185
2700 / 96185
2800 / 96185
2900 / 96185
3000 / 96185
3100 / 96185
3200 / 96185
3300 / 96185
3400 / 96185
3500 / 96185
3600 / 96185
3700 / 96185
3800 / 96185
3900 / 96185
4000 / 96185
4100 / 96185
4200 / 96185
4300 / 96185
4400 / 96185
4500 / 96185
4600 / 96185
4700 / 96185
4800 / 96185
4900 / 96185
5000 / 96185
5100 / 96185
5200 / 96185
5300 / 96185
5400 / 96185
5500 / 96185
5600 / 96185
5700 / 96185
5800 / 96185
5900 / 96185
6000 / 96185
6100 / 96185
6200 / 96185
6300 / 96185
6400 / 96185
6500 / 96185
6600 / 96185
6700 / 96185
6800 / 96185
6900 / 96185
7000 / 96185
7100 / 96185
7200 / 96185
7300 / 96185
7400 / 96185
7500 / 96185
7600 / 96185
7700 / 9618

59400 / 96185
59500 / 96185
59600 / 96185
59700 / 96185
59800 / 96185
59900 / 96185
60000 / 96185
60100 / 96185
60200 / 96185
60300 / 96185
60400 / 96185
60500 / 96185
60600 / 96185
60700 / 96185
60800 / 96185
60900 / 96185
61000 / 96185
61100 / 96185
61200 / 96185
61300 / 96185
61400 / 96185
61500 / 96185
61600 / 96185
61700 / 96185
61800 / 96185
61900 / 96185
62000 / 96185
62100 / 96185
62200 / 96185
62300 / 96185
62400 / 96185
62500 / 96185
62600 / 96185
62700 / 96185
62800 / 96185
62900 / 96185
63000 / 96185
63100 / 96185
63200 / 96185
63300 / 96185
63400 / 96185
63500 / 96185
63600 / 96185
63700 / 96185
63800 / 96185
63900 / 96185
64000 / 96185
64100 / 96185
64200 / 96185
64300 / 96185
64400 / 96185
64500 / 96185
64600 / 96185
64700 / 96185
64800 / 96185
64900 / 96185
65000 / 96185
65100 / 96185
65200 / 96185
65300 / 96185
65400 / 96185
65500 / 96185
65600 / 96185
65700 / 96185
65800 / 96185
65900 / 96185
66000 / 96185
66100 / 96185
66200 / 96185
66300 / 96185
66400 / 96185
66500 

In [17]:
print('q:', train_data[0][1])
print('kv:', train_kv_indices[0])
print(kv_pairs[4162], kv_pairs[705])

q: ['what', 'movies', 'are', 'about', 'ginger', 'rogers']
kv: [4162, 705]
['primrose path', 'starred_actors', 'ginger rogers', 'joel mccrea', 'marjorie rambeau'] ['daughters of the dust', 'starred_actors', 'cora lee day', 'trula hoosier', 'alva rogers']


In [18]:
save_pickle(train_kv_indices, 'mov_train_kv_indices.pickle')
save_pickle(test_kv_indices, 'mov_test_kv_indices.pickle')

save pickle to mov_train_kv_indices.pickle
save pickle to mov_test_kv_indices.pickle


In [36]:
n_entity = 8625
print(n_entity)
tf.get_variable_scope().reuse_variables()
B = tf.get_variable('B', shape=[500, n_entity], initializer=tf.contrib.layers.xavier_initializer())

8625


In [42]:
identity_mat = tf.diag(tf.ones(shape=[n_entity]))
y_data = np.zeros((n_entity, n_entity), dtype=np.float32)
for i in range(n_entity):
    y_data[i][i] = 1
# print(y_data[:10])

In [43]:
mul = tf.matmul(B, y_data)

In [44]:
mul.shape

TensorShape([Dimension(500), Dimension(8625)])

## vocab by ngram

In [33]:
train_stories = load_pickle('mov_task1_qa_pipe_train.pickle')
test_stories = load_pickle('mov_task1_qa_pipe_test.pickle')
entities = load_pickle('mov_entities.pickle')

load mov_task1_qa_pipe_train.pickle
load mov_task1_qa_pipe_test.pickle
load mov_entities.pickle


In [52]:
questions = [q for _, q, _ in train_stories]
print('questions:', questions[:2])
max_entity_length = max(map(len, (e.split(' ') for e in entities)))
print('max_entity_length:', max_entity_length)

questions: [['what', 'movies', 'are', 'about', 'ginger', 'rogers'], ['which', 'movies', 'can', 'be', 'described', 'by', 'moore']]
max_entity_size: 20


In [55]:
def find_ngrams(token_dict, text, n):
    """ See: https://github.com/facebookresearch/ParlAI/blob/master/parlai/core/dict.py#L31
        token_dict:  {'hello world', 'ol boy'}
        text: ['hello', 'world', 'buddy', 'ol', 'boy']
        n: max n of n-gram
        ret: ['hello world', 'buddy', 'ol boy']
    """
    """Breaks text into ngrams that appear in ``token_dict``."""
    # base case
    if n <= 1:
        return text
    # tokens committed to output
    saved_tokens = []
    # tokens remaining to be searched in sentence
    search_tokens = text[:]
    # tokens stored until next ngram found
    next_search = []
    while len(search_tokens) >= n:
        ngram = ' '.join(search_tokens[:n])
        if ngram in token_dict:
            # first, search previous unmatched words for smaller ngrams
            sub_n = min(len(next_search), n - 1)
            saved_tokens.extend(find_ngrams(token_dict, next_search, sub_n))
            next_search.clear()
            # then add this ngram
            saved_tokens.append(ngram)
            # then pop this ngram from the remaining words to search
            search_tokens = search_tokens[n:]
        else:
            next_search.append(search_tokens.pop(0))
    remainder = next_search + search_tokens
    sub_n = min(len(remainder), n - 1)
    saved_tokens.extend(find_ngrams(token_dict, remainder, sub_n))
    return saved_tokens

In [57]:
token_dict = set(entities)
for q in questions[:10]:
    res = find_ngrams(token_dict, q, max_entity_length)
    print(res)


['what', 'movies', 'are', 'about', 'ginger rogers']
['which', 'movies', 'can', 'be', 'described', 'by', 'moore']
['what', 'films', 'can', 'be', 'described', 'by', 'occupation']
['which', 'films', 'are', 'about', 'jacques tati']
['what', 'movies', 'are', 'about', 'donnie darko']
['what', 'movies', 'can', 'be', 'described', 'with', 'ben stiller']
['what', 'films', 'are', 'about', 'dillinger']
['what', 'movies', 'can', 'be', 'described', 'with', 'chris noonan']
['what', 'movies', 'are', 'about', 'george pal']
['what', 'movies', 'are', 'about', 'rod lurie']
